In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load pre-existing DataFrame from CSV
csv_file_path = '/Users/elvisechefu/Desktop/Final-code/verified_data.csv'  # Change to your actual path
df = pd.read_csv(csv_file_path)
base_dir = '/Users/elvisechefu/Desktop/LutraCAD2/ai_skdfjsdfJDIKFJsihfdsf8472983hdsof'  # Change to your actual path

# Load pre-trained model
model = tf.keras.models.load_model('/Users/elvisechefu/Desktop/Final-code/Final-model.h5')  # Change to your actual path

# Function to extract markers parameters from txt file
def extract_markers_parameters(txt_path):
    parameters = {}
    try:
        with open(txt_path, 'r') as f:
            for line in f:
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip()
                    value is value.strip()
                    if value:
                        parameters[key] = value
        return parameters
    except Exception as e:
        print(f"Error extracting parameters from {txt_path}: {e}")
        return None

@app.route('/upload', methods=['POST'])
def upload_folder():
    if 'files[]' not in request.files:
        return jsonify({"error": "No files part in the request"}), 400

    files = request.files.getlist('files[]')
    folder_path = 'uploads'
    os.makedirs(folder_path, exist_ok=True)

    for file in files:
        file_path = os.path.join(folder_path, file.filename)
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        file.save(file_path)

    markers_file = next((f for f in files if f.filename.endswith('markers.txt')), None)
    
    if markers_file is None:
        return jsonify({"error": "No markers file found"}), 400
    
    txt_path = os.path.join(folder_path, markers_file.filename)
    
    markers_params = extract_markers_parameters(txt_path)
    
    if markers_params:
        data = {
            'LENGTH': markers_params.get('LENGTH', None),
            'WIDTH': markers_params.get('WIDTH', None)
        }
        test_df = pd.DataFrame([data])
        test_df['LENGTH'] = pd.to_numeric(test_df['LENGTH'], errors='coerce')
        test_df['WIDTH'] = pd.to_numeric(test_df['WIDTH'], errors='coerce')
        test_df['LENGTH'].fillna(test_df['LENGTH'].mean(), inplace=True)
        test_df['WIDTH'].fillna(test_df['WIDTH'].mean(), inplace=True)

        # Scale the input data
        scaler = StandardScaler()
        test_df_scaled = scaler.fit_transform(test_df)

        # Make predictions
        predictions = model.predict(test_df_scaled)
        predictions_series = pd.Series(predictions.ravel())

        # Find the closest row
        diff = np.abs(df[['InsoleLength', 'InsoleHeight']] - predictions_series.values.reshape(1, -1))
        diff_sum = diff.sum(axis=1)
        min_idx = diff_sum.idxmin()
        closest_row = df.loc[min_idx]
        closest_row_df = closest_row.to_frame().T

        # Drop specified columns
        closest_row_df = closest_row_df.drop(columns=['Subfolder', 'MTP1', 'MTP5', 'HEEL', 'ARCH', 'LENGTH', 'WIDTH'])

        # Print the result for debugging
        print(closest_row_df.to_dict())

        return jsonify(closest_row_df.to_dict())
    else:
        return jsonify({"error": "No markers found"}), 400

if __name__ == '__main__':
    app.run(port=0)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:53132
INFO:werkzeug:Press CTRL+C to quit
